### N-gram Language Model
In this assignment, you will build several n-gram models. For submission, you will need to first ***make a copy of this notebook*** (File-> save a copy to drive). Next, fill out each of the functions for each task. Make sure to run each cell and check if the output is correct or not. Finally, submit a pdf (File->print). Note, you can not use any library for the code.

In [134]:
# import and download corpus
import nltk
from nltk.corpus import gutenberg

nltk.download('gutenberg')
nltk.download('punkt')

# getting a list of words from this book
caesar = gutenberg.words('shakespeare-caesar.txt')
# convert all to lower case
caesar = [l.lower() for l in caesar]
# print first ten words here
print(caesar[210:220])

[',', 'i', 'can', 'mend', 'you', 'mur', '.', 'what', 'mean', "'"]


[nltk_data] Downloading package gutenberg to /config/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /config/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Task 1: Write functions to count unique occurences and co-occurence. (Points: 5)



In [135]:
def count_unique_occurences(list_of_words):
  L = []
  # L should be a lit of tuples (a, b) where a is the word and b is the count
  # replace the list here with your code

  L = [(x, list_of_words.count(x)) for x in set(list_of_words)]

  return L


In [136]:
def count_co_occurences(list_of_words):
  #L = []
  # L should be a lit of tuples (a, b) where a is the word1:word2 and b is the count
  # replace the list here with your code
  #L = [('julius:ceasur', 5), ('i:am', 3)]

  counts = {}

  for i in range(len(list_of_words) - 1):
    key = f"{list_of_words[i]}:{list_of_words[i+1]}"
    if key in counts.keys():
      counts[key] += 1
    else:
      counts[key] = 1

  return list(counts.items())

In [137]:
# print first five occurences
occurences = count_unique_occurences(caesar)
_ = [print(l) for l in occurences[:5]]

('coffin', 1)
('goe', 4)
('signe', 4)
('defeat', 1)
('attyre', 1)


In [138]:
# print first five co-occurences
co_occurences = count_co_occurences(caesar)
_ = [print(l) for l in co_occurences[:5]]

('[:the', 1)
('the:tragedie', 2)
('tragedie:of', 2)
('of:julius', 1)
('julius:caesar', 1)


#### Task 2: write functions to compute unigram and bigram proability (Points: 5)

In [139]:
def occurences_probability(list_of_counts):
  L = []
  # L should be a lit of tuples (a, b) where a is the word and b is the probability
  # replace the list here with your code
  #L = [('julius', '0.2'), ('ceasur', '0.8')]

  total = sum(count for _, count in list_of_counts)

  L = [(word, count / total) for word, count in list_of_counts]

  return L

In [140]:
def co_occurences_probability(list_of_counts):
  L = []
  # L should be a lit of tuples (a, b) where a is the word1:word2 and b is the probability
  # replace the list here with your code

  # Build individual word count dict from pair word counts
  word_counts = {}
  for pair, count in list_of_counts:
    [a, b, *_] = pair.split(':')
    if a in word_counts.keys():
      word_counts[a] += count
    else:
      word_counts[a] = count
  # Add very last item in pair list (won't be added in for loop)
  last_pair, last_count = list_of_counts[-1]
  last_pair = last_pair.split(':')
  if last_pair[1] in word_counts.keys():
    word_counts[last_pair[1]] += last_count
  else:
    word_counts[last_pair[1]] = last_count

  pair_counts = dict(list_of_counts)

  # Populate L with the probability of pair[1] given pair[0] for each pair
  L = [(pair, pair_counts[pair] / word_counts[pair.split(':')[0]]) for pair, _ in list_of_counts]

  return L

In [158]:
# print first five occurences
occ_prob = occurences_probability(occurences)
_ = [print(l) for l in occ_prob[:5]]

('coffin', 3.871017690550846e-05)
('goe', 0.00015484070762203383)
('signe', 0.00015484070762203383)
('defeat', 3.871017690550846e-05)
('attyre', 3.871017690550846e-05)


In [142]:
# print first five co-occurences probabilities
co_occ_prob = co_occurences_probability(co_occurences)
_ = [print(l) for l in co_occ_prob[:5]]

('[:the', 0.3333333333333333)
('the:tragedie', 0.0034542314335060447)
('tragedie:of', 1.0)
('of:julius', 0.002824858757062147)
('julius:caesar', 1.0)


#### Task 3: Find probability of a sentence using unigram and bigram model (points 5)

In [143]:
def unigram_sent_probability(sent, prob_list):
  prob = 1
  # compute probability
  # here sent will be a string that you will have to split

  prob_dict = dict(prob_list)
  for word in sent.split(' '):
    prob *= prob_dict[word]

  return prob

In [144]:
def bigram_sent_probability(sent, prob_list):
  prob = 1
  # compute probability
  # here sent will be a string that you will have to split

  words = sent.split(' ')
  word_pairs = []
  for i in range(len(words) - 1):
    word_pairs.append(words[i] + ':' + words[i + 1])

  prob_dict = dict(prob_list)

  for pair in word_pairs:
    prob *= prob_dict[pair]

  return prob

In [145]:
sent = "i can mend you"
u_prob = unigram_sent_probability(sent, occ_prob)
b_prob = bigram_sent_probability(sent, co_occ_prob)
print("Unigram probability for", sent, "is:", u_prob)
print("Bigram probability for", sent, "is:", b_prob)

Unigram probability for i can mend you is: 1.7782310416861445e-11
Bigram probability for i can mend you is: 0.00039498370692208945


#### Task 4: Predict the most probable next word (Points 5)

In [146]:
def uni_predict_next_words(sent, prob_list, n=1):
  # go through all combination of words and find the next n words with highest probability and return the list
  # replace this with your code
  # word = ['I']

  prob_list.sort(key=lambda a: a[1], reverse=True)

  words = [word for word, _ in prob_list[:n]]
  return words


In [147]:
def bi_predict_next_words(sent, prob_list, n=1):
  # go through all combination of words and find the next n words with highest probability and return the list
  # replace this with your code
  # word = ['I']

  words = sent.split(' ')
  last_word = words[-1]
  prob_given_list = []
  for pair, prob in prob_list:
    [first, second, *_] = pair.split(':')
    if first == last_word:
      prob_given_list.append((second, prob))

  prob_given_list.sort(key=lambda a: a[1], reverse=True)

  words = [word for word, _ in prob_given_list[:n]]
  return words

In [148]:
u_next_words = uni_predict_next_words(sent, occ_prob, n=5)
b_next_words = bi_predict_next_words(sent, co_occ_prob, n=5)
print("Unigram-based next best word for", sent, "is:", u_next_words)
print("Bigram-based next best word for", sent, "is:", b_next_words)

Unigram-based next best word for i can mend you is: [',', '.', 'and', 'the', 'i']
Bigram-based next best word for i can mend you is: [',', 'are', 'know', 'haue', 'shall']


#### Task 5: Interpolated Probability. (Points 5)

In [149]:
def linearly_interpolated_sent_probability(sent, lam1, prob_list_u, lam2, prob_list_b):
  # here instead of uni or bigram, you will use linear interpolation to find the probability of the sentence
  # lambda values are lam1 for unigram and lam2 for bigram
  # replace this line with your code

  unigram_prob = unigram_sent_probability(sent, prob_list_u)
  bigram_prob = bigram_sent_probability(sent, prob_list_b)

  return lam1 * unigram_prob + lam2 * bigram_prob


In [150]:
# lets see how probability changes with lambda values
for i in range(1,10):
  l_prob = linearly_interpolated_sent_probability(sent, i/10, occ_prob, 1-(i)/10, co_occ_prob)
  print("With lamda ", i/10, "%.1f" % (1-(i)/10), "interpolated probability for", sent, "is:", l_prob)

With lamda  0.1 0.9 interpolated probability for i can mend you is: 0.0003554853380081116
With lamda  0.2 0.8 interpolated probability for i can mend you is: 0.0003159869690941337
With lamda  0.3 0.7 interpolated probability for i can mend you is: 0.0002764886001801557
With lamda  0.4 0.6 interpolated probability for i can mend you is: 0.0002369902312661778
With lamda  0.5 0.5 interpolated probability for i can mend you is: 0.00019749186235219993
With lamda  0.6 0.4 interpolated probability for i can mend you is: 0.00015799349343822205
With lamda  0.7 0.3 interpolated probability for i can mend you is: 0.00011849512452424414
With lamda  0.8 0.2 interpolated probability for i can mend you is: 7.899675561026621e-05
With lamda  0.9 0.1 interpolated probability for i can mend you is: 3.949838669628831e-05
